In [62]:
import math

import geojson
import requests
import tqdm

In [99]:
locations_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/locations'
nodes_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/nodes'
observation_types_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/observationTypes'
timeseries_url = 'http://pl-tc012.xtr.deltares.nl:8080/FewsWebServices/rest/digitaledelta/2.0/timeseries'

In [80]:
# get the first request and inspect it to get the paging information

def paging_iter(url):
    """iterate over all pages in a url"""
    resp = requests.get(url).json()
    page_size = resp['paging']['maxPageSize']
    count = resp['paging']['totalObjectCount']
    # Now loop over all pages
    n = math.ceil(count / page_size)

    # store each json result
    # progress please
    for i in tqdm.tqdm_notebook(range(n)):
        # fire the request. It's not faster in parallel (tested with asyncio), so I keep it serial.
        resp = requests.get(url, dict(page=i + 1, pageSize=page_size))
        yield resp.json()
    

In [105]:
locations_resps = list(paging_iter(locations_url))

In [106]:
# collect information about the information per location
# one request per location
timeseries_resps = list(paging_iter(timeseries_url))

In [120]:
# collect all the features
features = []
for resp in locations_resps:
    results = resp['results']
    for feature in results:
        feature['id'] = feature['properties']['locationId']
        features.append(feature)
        
features_by_id = {
    feature['id']: feature
    for feature 
    in features
}

timeseries = []
for resp in timeseries_resps:
    timeseries.extend(resp['results'])

In [140]:
for series in timeseries:
    id = series['location']['properties']['locationId']
    feature = features_by_id[id]
    quantities = feature['properties'].get('quantities', [])
    quantity = series['observationType']['quantity']
    if quantity not in quantities:
        quantities.append(quantity)
    feature['properties']['quantities'] = quantities
    features_by_id[feature['id']] = feature

In [141]:
# store the features
collection = geojson.FeatureCollection(features=list(features_by_id.values()))
with open('pl-tc012.geojson', 'w') as f:
    geojson.dump(collection, f)

In [142]:
# Create tiles until level 12 or something.... with tippecanoe

# let's upload it
!mapbox upload siggyf.pl-tc012 pl-tc012.geojson

Uploading data source  [####################################]  100%             
{"id":"cjxpw7j9n0lt22omw307oot3x","name":"pl-tc012","complete":false,"error":null,"created":"2019-07-05T09:24:48.599Z","modified":"2019-07-05T09:24:48.599Z","tileset":"siggyf.pl-tc012","owner":"siggyf","progress":0}
